In [5]:
from datetime import datetime, timedelta
from io import StringIO

import pandas as pd
from selenium import webdriver


def range_datetime(from_datetime, to_datetime):
    while from_datetime < to_datetime:
        yield from_datetime
        from_datetime += timedelta(days=1)



driver = webdriver.Chrome()
driver.implicitly_wait(30)

In [7]:
from_datetime = datetime.strptime('2024-01-01', '%Y-%m-%d')
to_datetime = datetime.strptime('2024-02-01', '%Y-%m-%d')

for date_time in range_datetime(from_datetime, to_datetime):
    datetime_str = date_time.strftime('%Y-%m-%d')
    driver.get("https://www.wunderground.com/history/daily/KLGA/date/" + datetime_str)

    table = driver.find_element('css selector', '.mat-table.cdk-table.mat-sort.ng-star-inserted')
    weather_df = pd.read_html(StringIO(table.get_attribute("outerHTML")))[0].dropna(subset=['Time'])

    weather_df['Time'] = weather_df['Time'].map(lambda r: datetime.strptime(datetime_str + ' ' + r, '%Y-%m-%d %I:%M %p'))
    weather_df.columns = weather_df.columns[1:].insert(0, 'Datetime')

    if from_datetime == date_time:
        weather_df.to_csv('userdata/NYC-weather-' + from_datetime.strftime('%Y-%m-%d') + '_' + to_datetime.strftime('%Y-%m-%d') + '.csv', header=True, index=False)
    else:
        weather_df.to_csv('userdata/NYC-weather-' + from_datetime.strftime('%Y-%m-%d') + '_' + to_datetime.strftime('%Y-%m-%d') + '.csv', header=False, index=False, mode='a')
    

In [8]:
driver.close()